# Mid Report - Flow

Limitations in current approaches:
1.   Lack of weather specific adaptation
2.   Lack of specialized consideration for pedestrain detection

Flow:
1.   **Genearte a synthetic snow dataset using KITTI**: augment clear-weather scenes with realistic snow effects: falling snowflakes and snowdrifts
2.   **Domain shift bewteen clear and snowy conditions**: integrates instance-level contrastive learning, DA-RAW - cluster pedestrains features across domains while separating dissimilar ones
3.   **Application of instance-level contrastive learning to focus on pedestrains**: leveraging contrastive losses

## CycleGAN and Pix2pix

In [ ]:
# Install Necessary Dependencies
# !pip install torch torchvision
# !pip install opencv-python-headless  # For OpenCV in Colab (headless version)


In [ ]:
# Clone the Repository
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix

# Install required dependencies
!pip install -r requirements.txt


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 16.86 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
# Download Pre-trained Models (Optional)
!bash scripts/download_cyclegan_model.sh summer2winter_yosemite

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [summer2winter_yosemite]
for details.

--2025-03-01 22:10:03--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/summer2winter_yosemite.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/summer2winter_yosemite_pretrained/latest_net_G.pth’

./checkpoints/summe 100%[===================>]  43.46M  12.7MB/s    in 3.8s    

2025-03-01 22:10:08 (11.5 MB/s) - ‘./checkpoints/summer2winter_yosemite_pretrained/latest_

In [ ]:
from google.colab import files

# Upload your image
uploaded = files.upload()

# Check the uploaded file's name
uploaded_image_path = next(iter(uploaded))
print(f"Uploaded file name: {uploaded_image_path}")


Saving 0000000000.png to 0000000000 (1).png
Uploaded file name: 0000000000 (1).png


In [ ]:
from PIL import Image
from torchvision import transforms

# Load the uploaded image
img = Image.open(uploaded_image_path)

# Resize and preprocess the image
img = img.resize((256, 256))  # Resize to 256x256
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])
img_tensor = preprocess(img).unsqueeze(0).cuda()  # Add batch dimension and move to GPU


In [ ]:
# Set options for inference
opt = TestOptions().parse()  # This will parse all arguments from the command line
opt.dataroot = '/content/pytorch-CycleGAN-and-pix2pix/imgs'
  # Use any folder (just needs to exist)
opt.name = 'summer2winter_yosemite'  # Pre-trained model name (should be in the 'checkpoints' folder)
opt.model = 'cycle_gan'  # Model type (CycleGAN)
opt.gpu_ids = [0]  # Use GPU if available
opt.batch_size = 1  # Batch size for inference
opt.phase = 'test'  # Set to 'test' phase (no dropout)
opt.no_dropout = True  # Disable dropout during inference

# Create and setup the model
model = create_model(opt)
model.setup()

usage: colab_kernel_launcher.py [-h] --dataroot DATAROOT [--name NAME] [--gpu_ids GPU_IDS]
                                [--checkpoints_dir CHECKPOINTS_DIR] [--model MODEL]
                                [--input_nc INPUT_NC] [--output_nc OUTPUT_NC] [--ngf NGF]
                                [--ndf NDF] [--netD NETD] [--netG NETG] [--n_layers_D N_LAYERS_D]
                                [--norm NORM] [--init_type INIT_TYPE] [--init_gain INIT_GAIN]
                                [--no_dropout] [--dataset_mode DATASET_MODE]
                                [--direction DIRECTION] [--serial_batches]
                                [--num_threads NUM_THREADS] [--batch_size BATCH_SIZE]
                                [--load_size LOAD_SIZE] [--crop_size CROP_SIZE]
                                [--max_dataset_size MAX_DATASET_SIZE] [--preprocess PREPROCESS]
                                [--no_flip] [--display_winsize DISPLAY_WINSIZE] [--epoch EPOCH]
                                [

SystemExit: 2

# First Version

## OpenCV

In [8]:
import cv2
import numpy as np

def opencv_add_snow_effect(image, num_snowflakes=1000, snowdrift_area=None, snowdrift_intensity=0.2):
    """
    Adds snowflakes and optionally a snowdrift effect to an image.

    Parameters:
    - image: The input image to which the snow effect will be applied.
    - num_snowflakes: The number of snowflakes to generate.
    - snowdrift_area: Optional mask area (tuple) for snowdrift (x, y, w, h).
    - snowdrift_intensity: Intensity of the snowdrift overlay.

    Returns:
    - snow_image: The final image with snowflakes and/or snowdrift.
    """
    if image is None:
        raise ValueError("The image could not be loaded. Please check the image path.")

    # Create a blank snow layer
    snow_layer = np.zeros_like(image, dtype=np.uint8)

    # Generate random snowflakes
    for _ in range(num_snowflakes):
        # Randomly generate position, size, and intensity of each snowflake
        x, y = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0])
        size = np.random.randint(1, 4)
        intensity = np.random.randint(200, 255)

        # Draw snowflake (small white circle)
        cv2.circle(snow_layer, (x, y), size, (intensity, intensity, intensity), -1)

    # Overlay snowflakes onto the original image (with transparency)
    alpha = 0.2
    snow_image = cv2.addWeighted(image, 1 - alpha, snow_layer, alpha, 0)

    # Add snowdrift effect if specified
    if snowdrift_area:
        # Create snowdrift in the specified area
        x, y, w, h = snowdrift_area
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)
    else:
        # Create random snowdrifts on random locations in the image
        x, y, w, h = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0]), 100, 50
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)

    return snow_image

# Example usage:
image_path = 'input.jpg'  # Update with the correct path to your image
image = cv2.imread(image_path)

if image is not None:
    snow_image = opencv_add_snow_effect(image)
    cv2.imwrite('output.jpg', snow_image)
else:
    print(f"Error: Could not load image at {image_path}")


## Diffusion Model

In [5]:
!pip install diffusers
!pip install torch
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [7]:
from google.colab import files
import shutil

uploaded = files.upload()

# Load your uploaded image
input_path = list(uploaded.keys())[0]  # This gets the filename of the uploaded image
shutil.move(input_path, "input.jpg")
print(f"✅ Image uploaded as 'input.jpg'")

# # Optionally, resize your image if needed (if too large)
# # input_image = input_image.resize((512, 512))


Saving 0000000000.png to 0000000000 (1).png
✅ Image uploaded as 'input.jpg'


In [9]:
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Step 3: Load the image
input_image = Image.open("output.jpg")  # Open the image using PIL

# # Optional: Display the image inline in the notebook using matplotlib
# plt.figure(figsize=(6, 6))  # Optional: Adjust the size of the display
# plt.imshow(input_image)
# plt.axis('off')  # Hide axis for better display
# plt.show()

# Step 4: Load the pre-trained Stable Diffusion Img2Img model
pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

# Step 5: Apply the Stable Diffusion Img2Img model to generate the snowy scene
prompt = "A beautiful snowy winter scene, soft falling snow, white landscape, high realism"
snowy_image = pipe(prompt=prompt, image=input_image, strength=0.5, guidance_scale=7.5).images[0]

# Step 6: Display the resulting image
snowy_image.show()

# Step 7: Save the output image
snowy_image.save("output_image.png")
print(f"✅ Output image saved as 'output_image.png'")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

✅ Output image saved as 'output_image.png'


In [10]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionImg2ImgPipeline

def diffusion_addsnow(input_image_path, output_image_path, prompt="A beautiful snowy winter scene, soft falling snow, white landscape, high realism", strength=0.5, guidance_scale=7.5):
    """
    Apply a snowy effect to an image using the Stable Diffusion Img2Img model.

    Parameters:
    - input_image_path: Path to the input image.
    - output_image_path: Path to save the output image.
    - prompt: Text prompt for the snow effect (default is for a snowy winter scene).
    - strength: How strongly to apply the effect (0.0 to 1.0).
    - guidance_scale: How strongly to guide the model by the prompt (higher values result in more prompt adherence).

    Returns:
    - None: Saves the generated snowy image to the specified output path.
    """
    # Step 1: Load the input image
    input_image = Image.open(input_image_path)  # Open the image using PIL

    # # Convert OpenCV image (numpy.ndarray) to PIL image
    # pil_image = Image.fromarray(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))

    # # Optional: Display the image inline in the notebook using matplotlib
    # plt.figure(figsize=(6, 6))  # Optional: Adjust the size of the display
    # plt.imshow(input_image)
    # plt.axis('off')  # Hide axis for better display
    # plt.show()

    # Step 2: Load the pre-trained Stable Diffusion Img2Img model
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

    # Step 3: Apply the Stable Diffusion Img2Img model to generate the snowy scene
    snowy_image = pipe(prompt=prompt, image=input_image, strength=strength, guidance_scale=guidance_scale).images[0]

    # Step 4: Display the resulting image
    snowy_image.show()

    # Step 5: Save the output image
    snowy_image.save(output_image_path)
    print(f"✅ Output image saved as '{output_image_path}'")


In [7]:
from google.colab import drive
drive.mount('/content/drive')

umount: /content/drive: not mounted.


ValueError: Mountpoint must not already contain files

In [13]:
import os
from PIL import Image
import cv2
import tempfile  # Importing the tempfile module

# Define paths
input_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02/'
output_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

# # Iterate over the images
# for image_file in image_files:
#     input_image_path = os.path.join(input_folder, image_file)
#     output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

#     # Step 1: Apply the OpenCV snow effect
#     input_image = cv2.imread(input_image_path)
#     snow_image_opencv = opencv_add_snow_effect(input_image)

#     # Step 2: Apply the Stable Diffusion snow effect
#     diffusion_addsnow(input_image_path, output_image_path)

#     print(f"Processed and saved: {output_image_path}")

# # Iterate over the images
# for image_file in image_files:
#     input_image_path = os.path.join(input_folder, image_file)
#     output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

#     # Step 1: Apply the OpenCV snow effect
#     input_image = cv2.imread(input_image_path)
#     snow_image_opencv = opencv_add_snow_effect(input_image)

#     # Step 2: Apply the Stable Diffusion snow effect using the OpenCV processed image
#     # Save only the final result
#     diffusion_addsnow(snow_image_opencv, output_image_path)

#     print(f"Processed and saved: {output_image_path}")

# # Iterate over the images
# for image_file in image_files:
#     input_image_path = os.path.join(input_folder, image_file)
#     output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

#     # Step 1: Apply the OpenCV snow effect
#     input_image = cv2.imread(input_image_path)
#     snow_image_opencv = opencv_add_snow_effect(input_image)

#     # Convert the OpenCV snow effect image (NumPy array) to a PIL image
#     snow_image_pil = Image.fromarray(cv2.cvtColor(snow_image_opencv, cv2.COLOR_BGR2RGB))

#     # Step 2: Apply the Stable Diffusion snow effect using the OpenCV processed image
#     diffusion_addsnow(snow_image_pil, output_image_path)

#     print(f"Processed and saved: {output_image_path}")

# Iterate over the images in your folder
for image_file in image_files:
    input_image_path = os.path.join(input_folder, image_file)
    output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

    # Step 1: Apply the OpenCV snow effect
    input_image = cv2.imread(input_image_path)
    snow_image_opencv = opencv_add_snow_effect(input_image)

    # Convert the OpenCV snow effect image (NumPy array) to a PIL image
    snow_image_pil = Image.fromarray(cv2.cvtColor(snow_image_opencv, cv2.COLOR_BGR2RGB))

    # Save the PIL image to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_file:
        temp_file_path = temp_file.name
        snow_image_pil.save(temp_file_path)

    # Step 2: Apply the Stable Diffusion snow effect using the file path
    diffusion_addsnow(temp_file_path, output_image_path)

    print(f"Processed and saved: {output_image_path}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

✅ Output image saved as '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000000_snow.png'
Processed and saved: /content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000000_snow.png


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

✅ Output image saved as '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000008_snow.png'
Processed and saved: /content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000008_snow.png


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

# First Version Full Code

## Full Code

In [14]:
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionImg2ImgPipeline
import os
from PIL import Image
import cv2
import tempfile  # Importing the tempfile module


def opencv_add_snow_effect(image, num_snowflakes=1000, snowdrift_area=None, snowdrift_intensity=0.2):
    """
    Adds snowflakes and optionally a snowdrift effect to an image.

    Parameters:
    - image: The input image to which the snow effect will be applied.
    - num_snowflakes: The number of snowflakes to generate.
    - snowdrift_area: Optional mask area (tuple) for snowdrift (x, y, w, h).
    - snowdrift_intensity: Intensity of the snowdrift overlay.

    Returns:
    - snow_image: The final image with snowflakes and/or snowdrift.
    """
    if image is None:
        raise ValueError("The image could not be loaded. Please check the image path.")

    # Create a blank snow layer
    snow_layer = np.zeros_like(image, dtype=np.uint8)

    # Set seed: to keep consisency as our input is a sequence of images
    np.random.seed(0)

    # Generate random snowflakes
    for _ in range(num_snowflakes):
        # Randomly generate position, size, and intensity of each snowflake
        x, y = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0])
        size = np.random.randint(1, 4)
        intensity = np.random.randint(200, 255)

        # Draw snowflake (small white circle)
        cv2.circle(snow_layer, (x, y), size, (intensity, intensity, intensity), -1)

    # Overlay snowflakes onto the original image (with transparency)
    alpha = 0.2
    snow_image = cv2.addWeighted(image, 1 - alpha, snow_layer, alpha, 0)

    # Add snowdrift effect if specified
    if snowdrift_area:
        # Create snowdrift in the specified area
        x, y, w, h = snowdrift_area
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)
    else:
        # Create random snowdrifts on random locations in the image
        x, y, w, h = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0]), 100, 50
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)

    return snow_image


def diffusion_addsnow(input_image_path, output_image_path, prompt="A beautiful snowy winter scene, soft falling snow, white landscape, high realism", strength=0.5, guidance_scale=7.5):
    """
    Apply a snowy effect to an image using the Stable Diffusion Img2Img model.

    Parameters:
    - input_image_path: Path to the input image.
    - output_image_path: Path to save the output image.
    - prompt: Text prompt for the snow effect (default is for a snowy winter scene).
    - strength: How strongly to apply the effect (0.0 to 1.0).
    - guidance_scale: How strongly to guide the model by the prompt (higher values result in more prompt adherence).

    Returns:
    - None: Saves the generated snowy image to the specified output path.
    """
    # Step 1: Load the input image
    input_image = Image.open(input_image_path)  # Open the image using PIL

    # # Convert OpenCV image (numpy.ndarray) to PIL image
    # pil_image = Image.fromarray(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))

    # # Optional: Display the image inline in the notebook using matplotlib
    # plt.figure(figsize=(6, 6))  # Optional: Adjust the size of the display
    # plt.imshow(input_image)
    # plt.axis('off')  # Hide axis for better display
    # plt.show()

    # Step 2: Load the pre-trained Stable Diffusion Img2Img model
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

    # Step 3: Apply the Stable Diffusion Img2Img model to generate the snowy scene
    snowy_image = pipe(prompt=prompt, image=input_image, strength=strength, guidance_scale=guidance_scale).images[0]

    # Step 4: Display the resulting image
    snowy_image.show()

    # Step 5: Save the output image
    snowy_image.save(output_image_path)
    print(f"✅ Output image saved as '{output_image_path}'")



# Define paths
input_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02/'
output_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

# Iterate over the images in your folder
for image_file in image_files:
    input_image_path = os.path.join(input_folder, image_file)
    output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

    # Step 1: Apply the OpenCV snow effect
    input_image = cv2.imread(input_image_path)
    snow_image_opencv = opencv_add_snow_effect(input_image)

    # Convert the OpenCV snow effect image (NumPy array) to a PIL image
    snow_image_pil = Image.fromarray(cv2.cvtColor(snow_image_opencv, cv2.COLOR_BGR2RGB))

    # Save the PIL image to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_file:
        temp_file_path = temp_file.name
        snow_image_pil.save(temp_file_path)

    # Step 2: Apply the Stable Diffusion snow effect using the file path
    diffusion_addsnow(temp_file_path, output_image_path)

    print(f"Processed and saved: {output_image_path}")

# Second Version: Currently Working on

## Rewrite above two functions, where random feature are excluded, as our input is a sequence of images

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import tempfile
from diffusers import StableDiffusionImg2ImgPipeline

def opencv_add_snow_effect(image, snowflakes, snowdrift_area=None, snowdrift_intensity=0.2):
    """
    Adds snowflakes to an image based on previous positions for consistency.

    Parameters:
    - image: The input image to which the snow effect will be applied.
    - snowflakes: A list of snowflakes with position and movement information.
    - snowdrift_area: Optional mask area (tuple) for snowdrift (x, y, w, h).
    - snowdrift_intensity: Intensity of the snowdrift overlay.

    Returns:
    - snow_image: The final image with snowflakes and/or snowdrift.
    """
    if image is None:
        raise ValueError("The image could not be loaded. Please check the image path.")

    # Create a blank snow layer
    snow_layer = np.zeros_like(image, dtype=np.uint8)

    # Draw each snowflake and update its position
    for flake in snowflakes:
        # Draw the snowflake
        cv2.circle(snow_layer, (int(flake['x']), int(flake['y'])),
                   flake['size'], (255, 255, 255), -1)

        # Update position for next frame
        flake['x'] += flake['dx']
        flake['y'] += flake['dy']

        # Reset snowflake if it goes out of bounds
        if flake['x'] >= image.shape[1]:
            flake['x'] = np.random.randint(0, 50)  # Reset to left side
            flake['y'] = np.random.randint(0, image.shape[0])
        elif flake['y'] >= image.shape[0] or flake['y'] < 0:
            flake['y'] = np.random.randint(0, image.shape[0])

    # Overlay snowflakes onto the original image (with transparency)
    alpha = 0.2
    snow_image = cv2.addWeighted(image, 1 - alpha, snow_layer, alpha, 0)

    # Add snowdrift effect if specified
    if snowdrift_area:
        x, y, w, h = snowdrift_area
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)
    else:
        x, y, w, h = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0]), 100, 50
        snowdrift = np.full_like(image, (255, 255, 255))
        snowdrift[y:y+h, x:x+w] = (255, 255, 255)
        snow_image = cv2.addWeighted(snow_image, 1, snowdrift, snowdrift_intensity, 0)

    return snow_image


def diffusion_addsnow(input_image_path, output_image_path, prompt="A beautiful snowy winter scene, soft falling snow, white landscape, high realism", strength=0.5, guidance_scale=7.5):
    """
    Apply a snowy effect to an image using the Stable Diffusion Img2Img model.

    Parameters:
    - input_image_path: Path to the input image.
    - output_image_path: Path to save the output image.
    - prompt: Text prompt for the snow effect (default is for a snowy winter scene).
    - strength: How strongly to apply the effect (0.0 to 1.0).
    - guidance_scale: How strongly to guide the model by the prompt (higher values result in more prompt adherence).

    Returns:
    - None: Saves the generated snowy image to the specified output path.
    """
    # Step 1: Load the input image
    input_image = Image.open(input_image_path)  # Open the image using PIL

    # Step 2: Load the pre-trained Stable Diffusion Img2Img model
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

    # Step 3: Apply the Stable Diffusion Img2Img model to generate the snowy scene
    snowy_image = pipe(prompt=prompt, image=input_image, strength=strength, guidance_scale=guidance_scale).images[0]

    # Step 4: Save the output image
    snowy_image.save(output_image_path)
    print(f"✅ Output image saved as '{output_image_path}'")





The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [15]:
# Define paths
input_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02/'
output_folder = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

# Initialize snowflakes list
snowflakes = []
snow_density = 400  # Adjust snow density
h, w = cv2.imread(os.path.join(input_folder, image_files[0])).shape[:2]
for _ in range(snow_density):
    x = np.random.randint(0, w)
    y = np.random.randint(0, h)
    size = np.random.randint(1, 4)
    dx = np.random.randint(3, 8)
    dy = np.random.randint(-1, 3)
    snowflakes.append({'x': x, 'y': y, 'size': size, 'dx': dx, 'dy': dy})

# Iterate over the images in your folder
for frame, image_file in enumerate(image_files):
    input_image_path = os.path.join(input_folder, image_file)
    output_image_path = os.path.join(output_folder, image_file.split('.')[0] + '_snow.png')

    # Step 1: Apply the OpenCV snow effect (with consistent snowflake movement)
    input_image = cv2.imread(input_image_path)
    snow_image_opencv = opencv_add_snow_effect(input_image, snowflakes)

    # Convert the OpenCV snow effect image (NumPy array) to a PIL image
    snow_image_pil = Image.fromarray(cv2.cvtColor(snow_image_opencv, cv2.COLOR_BGR2RGB))

    # Save the PIL image to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_file:
        temp_file_path = temp_file.name
        snow_image_pil.save(temp_file_path)

    # Step 2: Apply the Stable Diffusion snow effect using the file path
    diffusion_addsnow(temp_file_path, output_image_path)

    print(f"Processed and saved: {output_image_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

✅ Output image saved as '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000002_snow.png'
Processed and saved: /content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000002_snow.png


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

✅ Output image saved as '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000003_snow.png'
Processed and saved: /content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02_snow/0000000003_snow.png


In [14]:
import os
# Path to the folder
folder_path = '/content/drive/MyDrive/SYDE673/Images/2011_09_26_drive_0001_sync_image_02'

# Check if the folder exists
if os.path.exists(folder_path):
    print("The folder exists.")
else:
    print("The folder does not exist.")

The folder exists.


In [11]:
!fusermount -u /content/drive  # Unmount
!rm -rf /content/drive  # Remove any residual mount folders
from google.colab import drive
drive.mount('/content/drive')



fusermount: failed to unmount /content/drive: Invalid argument
Mounted at /content/drive


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
import os
folder_path = "/content/drive/MyDrive/SYDE673/Images"
print(os.listdir(folder_path))  # Should list the subfolders


['2011_09_26_drive_0001_sync_image_02', '2011_09_26_drive_0001_sync_image_02_snow']
